In [80]:
import pandas as pd
import re
import unicodedata
from pathlib import Path

In [81]:
# df = pd.read_excel('letterboxd.xlsx')
# df['URL'] = df['URL'].str.replace('https://boxd.it/', '')
# df['Year'] = df['Year'].astype(int)
# print(df.head())

In [82]:
def remove_accents(text):
    normalised = unicodedata.normalize('NFKD', text)
    return ''.join(c for c in normalised if not unicodedata.combining(c))

def clean_title(title):
    title = remove_accents(title)
    title = title.lower()
    title = re.sub(r'[^A-Za-z0-9 ]', '', title)

    if title[:4] == "the ":
        title = title[4:]
    elif title[:2] == "a ":
        title = title[2:]
    elif title[:3] == "an ":
        title = title[3:]
    
    return title
    


In [83]:
lists_folder = Path("lists")
csv_files = lists_folder.glob("*.csv")

dfs = []

for file in csv_files:
    df = pd.read_csv(file, skiprows=4)
    df['List'] = file.stem.replace('-', ' ')
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)
combined_df['URL'] = combined_df['URL'].str.replace('https://boxd.it/', '')
combined_df['Year'] = combined_df['Year'].astype(int)
combined_df['Sort Name'] = combined_df['Name'].apply(clean_title)

C:\Users\Tom\AppData\Local\Temp\ipykernel_37696\3675412222.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  combined_df['URL'] = combined_df['URL'].str.replace('https://boxd.it/', '')


In [84]:
combined_df.head()

,Position,Name,Year,URL,Description,List,Sort Name
0,1,All of Us Strangers,2023,Bz3C,NaN,1001 before you die,all of us strangers
1,2,The Zone of Interest,2023,gJsA,NaN,1001 before you die,zone of interest
2,3,The Boy and the Heron,2023,ipeM,NaN,1001 before you die,boy and the heron
3,4,Poor Things,2023,tNWU,NaN,1001 before you die,poor things
4,5,Oppenheimer,2023,wUow,NaN,1001 before you die,oppenheimer


In [85]:
film_counts = (
    combined_df.groupby("URL")
      .agg(
          appearances=("URL", "count"),
          name=("Name", "first"),
          year=("Year", "first"),
          lists=("List", list),
          sort=("Sort Name", "first")
      )
      .reset_index()
)

In [86]:
list_values_dict = {'1001 before you die' : 4,
                    'blank check discord' : 5,
                    'edgar wright' : 4,
                    'elgyem collection' : 2,
                    'empire masterpieces' : 4,
                    'empire top 100' : 2,
                    'guillermo del toro' : 3,
                    'imdb top 250' : 3,
                    'letterboxd top 250' : 2,
                    'movies everyone should watch' : 3,
                    'network favourites' : 1,
                    'sight and sound' : 5}

def list_value_calc(source_files):
    return sum(list_values_dict.get(f, 0.1) for f in source_files)

film_counts['list values'] = film_counts['lists'].apply(list_value_calc)

In [87]:
film_counts.head()

,URL,appearances,name,year,lists,sort,list values
0,10hC,2,Make Way for Tomorrow,1937,"[1001 before you die, elgyem collection]",make way for tomorrow,6
1,10iU,3,La Notte,1961,"[1001 before you die, edgar wright, letterboxd...",la notte,10
2,10iq,1,Pyaasa,1957,[sight and sound],pyaasa,5
3,10uQ,1,Terror Train,1980,[elgyem collection],terror train,2
4,11Hc,1,Baby Doll,1956,[elgyem collection],baby doll,2


In [88]:
film_count_counts = (
    film_counts["appearances"]
    .value_counts()
    .sort_index(ascending=False)
    .rename("number of films")
    .to_frame()
)

film_count_counts["cumulative total"] = film_count_counts["number of films"].cumsum()

print(film_count_counts)

    number of films  cumulative total
11                2                 2
10               11                13
9                23                36
8                30                66
7                52               118
6                44               162
5                79               241
4               125               366
3               237               603
2               500              1103
1              1661              2764


In [97]:
film_counts.head()

,URL,appearances,name,year,lists,sort,list values
0,10hC,2,Make Way for Tomorrow,1937,"[1001 before you die, elgyem collection]",make way for tomorrow,6
1,10iU,3,La Notte,1961,"[1001 before you die, edgar wright, letterboxd...",la notte,10
2,10iq,1,Pyaasa,1957,[sight and sound],pyaasa,5
3,10uQ,1,Terror Train,1980,[elgyem collection],terror train,2
4,11Hc,1,Baby Doll,1956,[elgyem collection],baby doll,2


In [101]:
list_length = 250

filtered_films = film_counts.sort_values(by=["appearances", "list values"], ascending=False)
filtered_films = filtered_films.head(list_length).sort_values('sort').reset_index()

for i, row in filtered_films.iterrows():
    print(f"{i+1}. {row['name']} ({row['year']}) - {row['appearances']}")

1. 12 Angry Men (1957) - 8
2. 2001: A Space Odyssey (1968) - 10
3. The 400 Blows (1959) - 7
4. 8½ (1963) - 8
5. Ace in the Hole (1951) - 5
6. Airplane! (1980) - 6
7. Akira (1988) - 6
8. Alien (1979) - 10
9. Aliens (1986) - 6
10. All About Eve (1950) - 8
11. All About My Mother (1999) - 5
12. All That Heaven Allows (1955) - 5
13. All That Jazz (1979) - 7
14. Amadeus (1984) - 8
15. Andrei Rublev (1966) - 5
16. Annie Hall (1977) - 5
17. The Apartment (1960) - 8
18. Apocalypse Now (1979) - 8
19. Army of Shadows (1969) - 5
20. Back to the Future (1985) - 8
21. Badlands (1973) - 4
22. Barry Lyndon (1975) - 9
23. The Battle of Algiers (1966) - 8
24. Beau Travail (1999) - 4
25. Before Sunrise (1995) - 6
26. Before Sunset (2004) - 5
27. Being John Malkovich (1999) - 5
28. Being There (1979) - 5
29. The Best Years of Our Lives (1946) - 5
30. Bicycle Thieves (1948) - 8
31. The Big Lebowski (1998) - 6
32. The Birds (1963) - 5
33. Blade Runner (1982) - 10
34. The Blair Witch Project (1999) - 5
35. 

In [47]:
selection = film_counts[film_counts["appearances"] == 7].sort_values(by="name", ascending=True)
for _, row in selection.iterrows():
    print(f"{row['name']} ({row['year']})")

A Clockwork Orange (1971)
A Matter of Life and Death (1946)
All That Jazz (1979)
Brazil (1985)
Bride of Frankenstein (1935)
Brief Encounter (1945)
City of God (2002)
Cléo from 5 to 7 (1962)
Come and See (1985)
Die Hard (1988)
Don't Look Now (1973)
Eternal Sunshine of the Spotless Mind (2004)
Everything Everywhere All at Once (2022)
Fanny and Alexander (1982)
Fargo (1996)
Fight Club (1999)
Get Out (2017)
Ghostbusters (1984)
High and Low (1963)
His Girl Friday (1940)
Inglourious Basterds (2009)
La Dolce Vita (1960)
Lawrence of Arabia (1962)
Magnolia (1999)
Metropolis (1927)
Network (1976)
No Country for Old Men (2007)
One Flew Over the Cuckoo's Nest (1975)
Oppenheimer (2023)
Persona (1966)
Rashomon (1950)
Schindler's List (1993)
Some Like It Hot (1959)
Sorcerer (1977)
Spider-Man: Into the Spider-Verse (2018)
Stalker (1979)
Suspiria (1977)
The 400 Blows (1959)
The Elephant Man (1980)
The Empire Strikes Back (1980)
The General (1926)
The Grand Budapest Hotel (2014)
The Piano (1993)
The Red

In [93]:
film_counts[film_counts['name'] == "La La Land"].head(1).lists.item()

['1001 before you die',
 'empire top 100',
 'movies everyone should watch',
 'network favourites']

In [32]:
film_counts[film_counts['name'].str.contains("Twin Peaks")]

,URL,appearances,name,year,lists,sort
1171,27Qw,4,Twin Peaks: Fire Walk with Me,1992,"[blank check discord, edgar wright, letterboxd...",twin peaks fire walk with me
2280,hN9O,1,Twin Peaks: The Return,2017,[sight and sound],twin peaks the return
